# Question1 
Re-analyse the annual maximum gust wind speeds for (I) the years 1952 to 1998, (II) the years 1952 to 1997, i.e. ignore the high value recorded in 1998. Compare the resulting predictions of design wind speeds for (a) 50 years return period, and (b) 1000 years return period, and comment.
# Question2 
Using the parameter estimation approach in code GB 50009 2012, predict the 10, 20, 50, 100, 200, 500, 1000 years return period design wind speeds for the above two cases, and compare the results with the Gumbel approach.


In [1]:

# 导入必要的库，pandas用于数据处理
import pandas as pd
from io import StringIO # python3中StringIO的位置

# 加载数据
data = """
year,Wind speed m/s
1952,31.4
1953,33.4
1954,29.8
1955,30.3
1956,27.8
1957,30.3
1958,29.3
1959,36.5
1960,29.3
1961,27.3
1962,31.9
1963,28.8
1964,25.2
1965,27.3
1966,23.7
1967,27.8
1968,32.4
1969,27.8
1970,26.2
1971,30.9
1972,31.9
1973,27.3
1974,25.7
1975,32.9
1976,28.3
1977,27.3
1978,28.3
1979,28.3
1980,29.3
1981,27.8
1982,27.8
1983,30.9
1984,26.7
1985,30.3
1986,28.3
1987,30.3
1988,34
1989,28.8
1990,30.3
1991,27.3
1992,27.8
1993,28.8
1994,30.9
1995,26.2
1996,25.7
1997,24.7
1998,42.2
"""

# 将data数据转换为StringIO对象，再读取为DataFrame
df = pd.read_csv(StringIO(data)) 


# Answer1

In [ ]:
import pandas as pd
import math 
from scipy.stats import linregress
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d

#整理数据
data = """
year,Wind speed m/s
1952,31.4
1953,33.4
1954,29.8
1955,30.3
1956,27.8
1957,30.3
1958,29.3
1959,36.5
1960,29.3
1961,27.3
1962,31.9
1963,28.8
1964,25.2
1965,27.3
1966,23.7
1967,27.8
1968,32.4
1969,27.8
1970,26.2
1971,30.9
1972,31.9
1973,27.3
1974,25.7
1975,32.9
1976,28.3
1977,27.3
1978,28.3
1979,28.3
1980,29.3
1981,27.8
1982,27.8
1983,30.9
1984,26.7
1985,30.3
1986,28.3
1987,30.3
1988,34
1989,28.8
1990,30.3
1991,27.3
1992,27.8
1993,28.8
1994,30.9
1995,26.2
1996,25.7
1997,24.7
1998,42.2
"""
lines = data.strip().split('\n')
def process_data(include_1998):
    wind_speeds = []
    for line in lines[1:]:  
        parts = line.split(',')
        year = parts[0]
        speed = float(parts[1])
        if not include_1998 and year == '1998':
            continue
        wind_speeds.append(speed)
    return wind_speeds
wind_speeds_case1 = process_data(include_1998=True)
wind_speeds_case1_sorted = sorted(wind_speeds_case1)
wind_speeds_case2 = process_data(include_1998=False)
wind_speeds_case2_sorted = sorted(wind_speeds_case2)

##Gumbel方法预测风速

#求数据点reduced variate（Gumbel）
non_exceedance_probability_case1_Gumbel = [(i+1)/ (len(wind_speeds_case1_sorted)+1) for i in range(len(wind_speeds_case1_sorted))]
non_exceedance_probability_case2_Gumbel = [(i+1)/ (len(wind_speeds_case2_sorted)+1) for i in range(len(wind_speeds_case2_sorted))]
reduced_variate_case1_Gumbel = [-math.log(-math.log(p)) for p in non_exceedance_probability_case1_Gumbel]
reduced_variate_case1_Gumbel = [round(num, 3) for num in reduced_variate_case1_Gumbel]
reduced_variate_case2_Gumbel = [-math.log(-math.log(p)) for p in non_exceedance_probability_case2_Gumbel]
reduced_variate_case2_Gumbel = [round(num, 3) for num in reduced_variate_case2_Gumbel]

#情况Ⅰ线性回归（Gumbel）
slope1_Gumbel, intercept1_Gumbel, r_value1_Gumbel, p_value1_Gumbel, std_err1_Gumbel = linregress(reduced_variate_case1_Gumbel, wind_speeds_case1_sorted)
slope1_Gumbel = round(slope1_Gumbel, 2)
intercept1_Gumbel = round(intercept1_Gumbel, 2)
reduced_variate_case1_Gumbel = np.array(reduced_variate_case1_Gumbel)
y_case1_Gumbel = slope1_Gumbel * reduced_variate_case1_Gumbel + intercept1_Gumbel

#情况Ⅱ线性回归（Gumbel）
slope2_Gumbel, intercept2_Gumbel, r_value2_Gumbel, p_value2_Gumbel, std_err2_Gumbel = linregress(reduced_variate_case2_Gumbel, wind_speeds_case2_sorted)
slope2_Gumbel = round(slope2_Gumbel, 2)
intercept2_Gumbel = round(intercept2_Gumbel, 2)
reduced_variate_case2_Gumbel = np.array(reduced_variate_case2_Gumbel)
y_case2_Gumbel = slope2_Gumbel * reduced_variate_case2_Gumbel + intercept2_Gumbel

#计算50年重现期和1000年重现期reduced variate
non_exceedance_probability_50 = 1-1/50
reduced_variate_50 = -math.log(-math.log(non_exceedance_probability_50)) 
reduced_variate_50 = round(reduced_variate_50,3)
non_exceedance_probability_1000 = 1-1/1000
reduced_variate_1000 = -math.log(-math.log(non_exceedance_probability_1000)) 
reduced_variate_1000 = round(reduced_variate_1000,3)

#情况Ⅰ求50年重现期和1000年重现期设计风速（Gumbel）
case1_design_wind_speeds_50_Gumbel = reduced_variate_50 * slope1_Gumbel + intercept1_Gumbel
case1_design_wind_speeds_50_Gumbel = round(case1_design_wind_speeds_50_Gumbel,1)
case1_design_wind_speeds_1000_Gumbel= reduced_variate_1000 * slope1_Gumbel + intercept1_Gumbel
case1_design_wind_speeds_1000_Gumbel = round(case1_design_wind_speeds_1000_Gumbel,1)

#情况Ⅱ求50年重现期和1000年重现期设计风速（Gumbel）
case2_design_wind_speeds_50_Gumbel = reduced_variate_50 * slope2_Gumbel + intercept2_Gumbel
case2_design_wind_speeds_50_Gumbel = round(case2_design_wind_speeds_50_Gumbel,1)
case2_design_wind_speeds_1000_Gumbel= reduced_variate_1000 * slope2_Gumbel + intercept2_Gumbel
case2_design_wind_speeds_1000_Gumbel = round(case2_design_wind_speeds_1000_Gumbel,1)

##Gringorten方法预测风速

#求数据点reduced variate（Gringorten）
non_exceedance_probability_case1_Gringorten = [(i+1-0.44)/ (len(wind_speeds_case1_sorted)+0.12) for i in range(len(wind_speeds_case1_sorted))]
non_exceedance_probability_case2_Gringorten = [(i+1-0.44)/ (len(wind_speeds_case2_sorted)+0.12) for i in range(len(wind_speeds_case2_sorted))]
reduced_variate_case1_Gringorten = [-math.log(-math.log(p)) for p in non_exceedance_probability_case1_Gringorten]
reduced_variate_case1_Gringorten = [round(num, 3) for num in reduced_variate_case1_Gringorten]
reduced_variate_case2_Gringorten = [-math.log(-math.log(p)) for p in non_exceedance_probability_case2_Gringorten]
reduced_variate_case2_Gringorten = [round(num, 3) for num in reduced_variate_case2_Gringorten]

#情况Ⅰ线性回归（Gringorten）
slope1_Gringorten, intercept1_Gringorten, r_value1_Gringorten, p_value1_Gringorten, std_err1_Gringorten = linregress(reduced_variate_case1_Gringorten, wind_speeds_case1_sorted)
slope1_Gringorten = round(slope1_Gringorten, 2)
intercept1_Gringorten = round(intercept1_Gringorten, 2)
reduced_variate_case1_Gringorten = np.array(reduced_variate_case1_Gringorten)
y_case1_Gringorten = slope1_Gringorten * reduced_variate_case1_Gringorten + intercept1_Gringorten


#情况Ⅱ线性回归（Gringorten）
slope2_Gringorten, intercept2_Gringorten, r_value2_Gringorten, p_value2_Gringorten, std_err2_Gringorten = linregress(reduced_variate_case2_Gringorten, wind_speeds_case2_sorted)
slope2_Gringorten = round(slope2_Gringorten, 2)
intercept2_Gringorten = round(intercept2_Gringorten, 2)
reduced_variate_case2_Gringorten = np.array(reduced_variate_case2_Gringorten)
y_case2_Gringorten = slope2_Gringorten * reduced_variate_case2_Gringorten + intercept2_Gringorten


#情况Ⅰ求50年重现期和1000年重现期设计风速（Gringorten）
case1_design_wind_speeds_50_Gringorten = reduced_variate_50 * slope1_Gringorten + intercept1_Gringorten
case1_design_wind_speeds_50_Gringorten = round(case1_design_wind_speeds_50_Gringorten,1)
case1_design_wind_speeds_1000_Gringorten= reduced_variate_1000 * slope1_Gringorten + intercept1_Gringorten
case1_design_wind_speeds_1000_Gringorten = round(case1_design_wind_speeds_1000_Gringorten,1)

#情况Ⅱ求50年重现期和1000年重现期设计风速（Gringorten）
case2_design_wind_speeds_50_Gringorten = reduced_variate_50 * slope2_Gringorten + intercept2_Gringorten
case2_design_wind_speeds_50_Gringorten = round(case2_design_wind_speeds_50_Gringorten,1)
case2_design_wind_speeds_1000_Gringorten= reduced_variate_1000 * slope2_Gringorten + intercept2_Gringorten
case2_design_wind_speeds_1000_Gringorten = round(case2_design_wind_speeds_1000_Gringorten,1)

#绘制图像
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

axs[0,0].scatter(reduced_variate_case1_Gumbel, wind_speeds_case1_sorted, color='blue', label='Data point')
axs[0,0].plot(reduced_variate_case1_Gumbel, y_case1_Gumbel, color='red', label='Fitting curve')
axs[0,0].set_title('Case Ⅰ EAST SALE GUSTWIND SPEEDS 1952-1998')
axs[0,0].set_xlabel('reduced variate(Gumbel)')
axs[0,0].set_ylabel('wind speeds(m/s)')
equation1 = f'y = {slope1_Gumbel}x + {intercept1_Gumbel}'
axs[0,0].text(0.05, 0.95, equation1, transform=axs[0, 0].transAxes, fontsize=12, verticalalignment='top')
axs[0,0].legend()
axs[0,0].grid(True)

axs[0,1].scatter(reduced_variate_case2_Gumbel, wind_speeds_case2_sorted, color='blue', label='Data point')
axs[0,1].plot(reduced_variate_case2_Gumbel, y_case2_Gumbel, color='red', label='Fitting curve')
axs[0,1].set_title('Case Ⅱ EAST SALE GUSTWIND SPEEDS 1952-1997')
axs[0,1].set_xlabel('reduced variate(Gumbel)')
axs[0,1].set_ylabel('wind speeds(m/s)')
equation2 = f'y = {slope2_Gumbel}x + {intercept2_Gumbel}'
axs[0,1].text(0.05, 0.95, equation2, transform=axs[0, 1].transAxes, fontsize=12, verticalalignment='top')
axs[0,1].legend()
axs[0,1].grid(True)

axs[1,0].scatter(reduced_variate_case1_Gringorten, wind_speeds_case1_sorted, color='blue', label='Data point')
axs[1,0].plot(reduced_variate_case1_Gringorten, y_case1_Gringorten, color='red', label='Fitting curve')
axs[1,0].set_title('Case Ⅰ EAST SALE GUSTWIND SPEEDS 1952-1998')
axs[1,0].set_xlabel('reduced variate(Gringorten)')
axs[1,0].set_ylabel('wind speeds(m/s)')
equation3 = f'y = {slope1_Gringorten}x + {intercept1_Gringorten}'
axs[1,0].text(0.05, 0.95, equation3, transform=axs[1, 0].transAxes, fontsize=12, verticalalignment='top')
axs[1,0].legend()
axs[1,0].grid(True)

axs[1,1].scatter(reduced_variate_case2_Gringorten, wind_speeds_case2_sorted, color='blue', label='Data point')
axs[1,1].plot(reduced_variate_case2_Gringorten, y_case2_Gringorten, color='red', label='Fitting curve')
axs[1,1].set_title('Case Ⅱ EAST SALE GUSTWIND SPEEDS 1952-1997')
axs[1,1].set_xlabel('reduced variate(Gringorten)')
axs[1,1].set_ylabel('wind speeds(m/s)')
equation4 = f'y = {slope2_Gringorten}x + {intercept2_Gringorten}'
axs[1,1].text(0.05, 0.95, equation4, transform=axs[1, 1].transAxes, fontsize=12, verticalalignment='top')
axs[1,1].legend()
axs[1,1].grid(True)

plt.tight_layout()
plt.show()

print(f"情况Ⅰ(包含98年极值)Gumbel方法预测50年重现期设计风速: {case1_design_wind_speeds_50_Gumbel} m/s")
print(f"情况Ⅱ(不包含98年极值)Gumbel方法预测50年重现期设计风速: {case2_design_wind_speeds_50_Gumbel} m/s")
print(f"情况Ⅰ(包含98年极值)Gumbel方法预测1000年重现期设计风速: {case1_design_wind_speeds_1000_Gumbel} m/s")
print(f"情况Ⅱ(不包含98年极值)Gumbel方法预测1000年重现期设计风速: {case2_design_wind_speeds_1000_Gumbel} m/s")
print(f"情况Ⅰ(包含98年极值)Gringorten方法预测50年重现期设计风速: {case1_design_wind_speeds_50_Gringorten} m/s")
print(f"情况Ⅱ(不包含98年极值)Gringorten方法预测50年重现期设计风速: {case2_design_wind_speeds_50_Gringorten} m/s")
print(f"情况Ⅰ(包含98年极值)Gringorten方法预测1000年重现期设计风速: {case1_design_wind_speeds_1000_Gringorten} m/s")
print(f"情况Ⅱ(不包含98年极值)Gringorten方法预测1000年重现期设计风速: {case2_design_wind_speeds_1000_Gringorten} m/s")
print(f"比较：\n1.Gringorten方法在预测设计风速时，相比于Gumbel方法，预测的风速较小。")
print(f"2.无论是Gumbel方法还是Gringorten方法，当数据源包含98年极值数据时，预测结果都高于剔除98年极值时的结果。")
print(f"这说明两种方法对极值都比较敏感，极值对整体分布的估计影响较大，尤其是在预测长重现期的设计风速时。")
print(f"3.Gringorten方法在包含和不包含98年极值时的预测值相对Gumbel方法更稳定一些，波动幅度略小。")

# Answer2

In [ ]:

#情况Ⅰ
mean_case1 = np.mean(wind_speeds_case1_sorted)
std_dev_case1 = np.std(wind_speeds_case1_sorted)
n = np.array([45,50])
C1 = np.array([1.15185,1.16066])
C2 = np.array([0.54630,0.54853])
linear_interp1 = interp1d(n, C1, kind='linear')
C1_case1 = linear_interp1(len(wind_speeds_case1_sorted))
linear_interp2 = interp1d(n, C2, kind='linear')
C2_case1 = linear_interp2(len(wind_speeds_case1_sorted))
a_case1 = C1_case1 / std_dev_case1
u_case1 = mean_case1 - C2_case1 / a_case1
non_exceedance_probability_10 = 1 - 1 / 10
non_exceedance_probability_20 = 1 - 1 / 20
non_exceedance_probability_50 = 1 - 1 / 50
non_exceedance_probability_100 = 1 - 1 / 100
non_exceedance_probability_200 = 1 - 1 / 200
non_exceedance_probability_500 = 1 - 1 / 500
non_exceedance_probability_1000 = 1 - 1 / 1000
reduced_variate_10 = -math.log(-math.log(non_exceedance_probability_10))
reduced_variate_20 = -math.log(-math.log(non_exceedance_probability_20))
reduced_variate_50 = -math.log(-math.log(non_exceedance_probability_50))
reduced_variate_100 = -math.log(-math.log(non_exceedance_probability_100))
reduced_variate_200 = -math.log(-math.log(non_exceedance_probability_200))
reduced_variate_500 = -math.log(-math.log(non_exceedance_probability_500))
reduced_variate_1000 = -math.log(-math.log(non_exceedance_probability_1000))
case1_design_wind_speeds_10 = round(reduced_variate_10 / a_case1 + u_case1,1)
case1_design_wind_speeds_20 = round(reduced_variate_20 / a_case1 + u_case1,1)
case1_design_wind_speeds_50 = round(reduced_variate_50 / a_case1 + u_case1,1)
case1_design_wind_speeds_100 = round(reduced_variate_100 / a_case1 + u_case1,1)
case1_design_wind_speeds_200 = round(reduced_variate_200 / a_case1 + u_case1,1)
case1_design_wind_speeds_500 = round(reduced_variate_500 / a_case1 + u_case1,1)
case1_design_wind_speeds_1000 = round(reduced_variate_1000 / a_case1 + u_case1,1)

#情况Ⅱ
mean_case2 = np.mean(wind_speeds_case2_sorted)
std_dev_case2 = np.std(wind_speeds_case2_sorted)
C1_case2 = linear_interp1(len(wind_speeds_case2_sorted))
C2_case2 = linear_interp2(len(wind_speeds_case2_sorted))
a_case2 = C1_case2 / std_dev_case2
u_case2 = mean_case2 - C2_case2 / a_case2
case2_design_wind_speeds_10 = round(reduced_variate_10 / a_case2 + u_case2,1)
case2_design_wind_speeds_20 = round(reduced_variate_20 / a_case2 + u_case2,1)
case2_design_wind_speeds_50 = round(reduced_variate_50 / a_case2 + u_case2,1)
case2_design_wind_speeds_100 = round(reduced_variate_100 / a_case2 + u_case2,1)
case2_design_wind_speeds_200 = round(reduced_variate_200 / a_case2 + u_case2,1)
case2_design_wind_speeds_500 = round(reduced_variate_500 / a_case2 + u_case2,1)
case2_design_wind_speeds_1000 = round(reduced_variate_1000 / a_case2 + u_case2,1)

print(f"情况Ⅰ(包含98年极值)GB2012方法预测10年重现期设计风速: {case1_design_wind_speeds_10} m/s")
print(f"情况Ⅰ(包含98年极值)GB2012方法预测20年重现期设计风速: {case1_design_wind_speeds_20} m/s")
print(f"情况Ⅰ(包含98年极值)GB2012方法预测50年重现期设计风速: {case1_design_wind_speeds_50} m/s")
print(f"情况Ⅰ(包含98年极值)GB2012方法预测100年重现期设计风速: {case1_design_wind_speeds_100} m/s")
print(f"情况Ⅰ(包含98年极值)GB2012方法预测200年重现期设计风速: {case1_design_wind_speeds_200} m/s")
print(f"情况Ⅰ(包含98年极值)GB2012方法预测500年重现期设计风速: {case1_design_wind_speeds_500} m/s")
print(f"情况Ⅰ(包含98年极值)GB2012方法预测1000年重现期设计风速: {case1_design_wind_speeds_1000} m/s")

print(f"情况Ⅱ(不包含98年极值)GB2012方法预测10年重现期设计风速: {case2_design_wind_speeds_10} m/s")
print(f"情况Ⅱ(不包含98年极值)GB2012方法预测20年重现期设计风速: {case2_design_wind_speeds_20} m/s")
print(f"情况Ⅱ(不包含98年极值)GB2012方法预测50年重现期设计风速: {case2_design_wind_speeds_50} m/s")
print(f"情况Ⅱ(不包含98年极值)GB2012方法预测100年重现期设计风速: {case2_design_wind_speeds_100} m/s")
print(f"情况Ⅱ(不包含98年极值)GB2012方法预测200年重现期设计风速: {case2_design_wind_speeds_200} m/s")
print(f"情况Ⅱ(不包含98年极值)GB2012方法预测500年重现期设计风速: {case2_design_wind_speeds_500} m/s")
print(f"情况Ⅱ(不包含98年极值)GB2012方法预测1000年重现期设计风速: {case2_design_wind_speeds_1000} m/s")

print(f"GB2012方法比Gumbel方法预测的风速略高一些。同时，当有极端值时，预测风速会增大，尤其在预测长重现期的情况。")
print(f"GB2012方法在包含和不包含极值时的预测值相对Gumbel方法更稳定一些，波动幅度略小。")